### Importing libraries and functions

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
import sys
sys.path.append('../')
from scripts.scrape_oldlistings import scrape_postcodes, scrape_postcodes_from_file, get_oldlisting_data, get_remaining_suburbs, get_remaining_oldlisting_data, convert_csv_to_parquet
from scripts.preprocess_oldlistings import preprocess_olist, create_forecast_template, split_by_gcc, split_domain_by_gcc
# from scripts.preproccessing import combine_SA2

### Datascrape

In [2]:
#scrape_postcodes_from_file()

suburb postcode
0     Abbotsford     3067
1     Aberfeldie     3040
2      Aberfeldy     3825
3        Acheron     3714
4      Addington     3352
...          ...      ...
1698     Yeodene     3249
1699      Yinnar     3869
1700   Youanmite     3646
1701     Yundool     3727
1702      Yuroke     3063

[1703 rows x 2 columns]

In [3]:
#get_oldlisting_data()

Getting data for Abbotsford, 3067. Suburb 0 of 1703
num pages: 96
Getting data for Aberfeldie, 3040. Suburb 1 of 1703
num pages: 11
Getting data for Aberfeldy, 3825. Suburb 2 of 1703
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Aberfeldy/3825/rent/. Continuing with other URLs.
Getting data for Acheron, 3714. Suburb 3 of 1703
num pages: 1
Getting data for Addington, 3352. Suburb 4 of 1703
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Addington/3352/rent/. Continuing with other URLs.
Getting data for Agnes, 3962. Suburb 5 of 1703
num pages: 1
Getting data for Aireys+Inlet, 3231. Suburb 6 of 1703
num pages: 5
Getting data for Airport+West, 3042. Suburb 7 of 1703
num pages: 45
Getting data for Albanvale, 3021. Suburb 8 of 1703
num pages: 12
Getting data for Albert+Park, 3206. Suburb 9 of 1703
num pages: 47
Getting data for Alberton, 3971. Suburb 10 of 1703
num pages: 1
Getting data for Albion, 3020. Suburb 11 of 1703
num pages: 25
Getting data f

In [2]:
#get_remaining_suburbs()

In [9]:
#get_remaining_oldlisting_data()

Getting data for West+Melbourne, 3003. Suburb 0 of 97
num pages: 106
Getting data for Westmeadows, 3049. Suburb 1 of 97
num pages: 22
Getting data for Westmere, 3351. Suburb 2 of 97
num pages: 1
Getting data for Wheelers+Hill, 3150. Suburb 3 of 97
num pages: 48
Getting data for White+Hills, 3550. Suburb 4 of 97
num pages: 15
Getting data for Whitfield, 3733. Suburb 5 of 97
num pages: 1
Getting data for Whitlands, 3733. Suburb 6 of 97
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Whitlands/3733/rent/. Continuing with other URLs.
Getting data for Whittington, 3219. Suburb 7 of 97
num pages: 14
Getting data for Whittlesea, 3757. Suburb 8 of 97
num pages: 14
Getting data for Whorouly, 3735. Suburb 9 of 97
num pages: 2
Getting data for Wickliffe, 3379. Suburb 10 of 97
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Wickliffe/3379/rent/. Continuing with other URLs.
Getting data for Wilby, 3728. Suburb 11 of 97
num pages: 1
Getting data for Willaura, 

### Initial handling of the data

In [3]:
convert_csv_to_parquet()

In [2]:
spark = (
    SparkSession.builder.appName("Preliminary Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/25 12:22:06 WARN Utils: Your hostname, LAPTOP-UFNFC5IK resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/25 12:22:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 12:22:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Preprocessing the data

In [3]:
# sdf = spark.read.parquet("../data/curated/properties.parquet")
# sdf.show()
# pandas = sdf.toPandas()

# combined = combine_SA2(pandas)
# print(combined.head())
#split_by_gcc(spark)
split_domain_by_gcc(spark)

KeyError: ['longitude']

In [1]:
import pandas as pd
# sdf = spark.read.parquet("../data/raw/oldlisting/greater_melb_properties_unprocessed.csv")
df = pd.read_csv("../data/raw/oldlisting/rest_of_vic_properties_unprocessed.csv")
df.count()

Unnamed: 0    21039
suburb        21039
postcode      21039
address       21039
latitude      21039
longitude     21039
beds          20263
baths         20240
cars          16857
house_type    14849
dates         21039
price_str     21039
SA2_CODE21    21039
GCC_NAME21    21039
dtype: int64

In [4]:
preprocess_olist(spark)

+-----------------+--------+--------------------+--------------+--------------+----+-----+----+-------------+----+------------+
|           suburb|postcode|             address|      latitude|     longitude|beds|baths|cars|property_type|date|weekly_price|
+-----------------+--------+--------------------+--------------+--------------+----+-----+----+-------------+----+------------+
|      albert park|    3206|  21 faussett street|    -37.842008|    144.955458| 2.0|  1.0| 1.0|        house|2024|       680.0|
|      albert park|    3206|  21 faussett street|    -37.842008|    144.955458| 2.0|  1.0| 1.0|        house|2024|       775.0|
|      albert park|    3206|  21 faussett street|    -37.842008|    144.955458| 2.0|  1.0| 1.0|        house|2023|       775.0|
|      albert park|    3206|  21 faussett street|    -37.842008|    144.955458| 2.0|  1.0| 1.0|        house|2021|       680.0|
|      albert park|    3206|  21 faussett street|    -37.842008|    144.955458| 2.0|  1.0| 1.0|        h

24/09/20 16:31:33 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2244368 ms exceeds timeout 120000 ms
24/09/20 16:31:33 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/20 16:31:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [3]:
create_forecast_template(spark)

118038
27989


In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, from_json, col, regexp_replace, regexp_extract, split, when, expr, trim, lower
from pyspark.sql.types import ArrayType, StringType, IntegerType

df = sdf.withColumn("price_str", regexp_replace("price_str", "'", '"'))

# Define the schema of the array inside the string
array_schema = ArrayType(StringType())

# Convert the string to an actual array using from_json
df = df.withColumn("price_str", from_json("price_str", array_schema))

# Explode the 'costs' array to flatten it
df_flattened = df.withColumn("ind_price_str", explode("price_str"))

# Select distinct costs
distinct_costs = df_flattened.select("ind_price_str").distinct()

# Define patterns
range_pattern = r'(\$\d{1,3}(?:,\d{3})*|\d+)\s*-\s*(\$\d{1,3}(?:,\d{3})*|\d+)'  # To correctly capture ranges with or without commas and currency symbols
# price_pattern = r'\$?(\d{1,3}(?:,\d{3})*|\d+)'  # To capture single prices
price_pattern = r'\$?(\d+(?:,\d{3})*|\d+)'
suffix_pattern = r'\s+([a-zA-Z\s]+)$'  # Capture suffixes that are words at the end of the string

df_processed = df_flattened.withColumn("range", regexp_extract("ind_price_str", range_pattern, 0)) \
                 .withColumn("single_price", regexp_extract("ind_price_str", price_pattern, 0)) \
                 .withColumn("avg_price", when(col("range") != "",
                                               (expr("cast(regexp_replace(split(range, '-')[0], '[\$,]', '') as double)") +
                                                expr("cast(regexp_replace(split(range, '-')[1], '[\$,]', '') as double)")) / 2)
                                          .otherwise(expr("cast(regexp_replace(single_price, '[\$,]', '') as double)"))) \
                 .withColumn("suffix", trim(regexp_extract("ind_price_str", suffix_pattern, 1)))

# Define a classification of suffixes based on keywords
df_classified = df_processed.withColumn("classification", 
                             when(col("suffix") == "", when(col("avg_price") >= 50000, "likely sale").otherwise("week"))
                              .when(lower(col("suffix")).rlike("(?<!million)week|pw|wk"), "week")
                              .when(lower(col("suffix")).rlike("month|pcm"), "month")
                              .when(lower(col("suffix")).rlike("annum|pa|annual"), "year")
                              .when(lower(col("suffix")).rlike("season|seasonally"), "season")
                              .otherwise("other"))

df_adjusted = df_classified.withColumn("weekly_price",
                            when(col("classification") == "week", col("avg_price"))
                            .when(col("classification") == "month", col("avg_price") / 4.3)
                            .when(col("classification") == "year", col("avg_price") / 52)
                            .when(col("classification") == "season", col("avg_price") / 13)
                            )

# Show results
COLS_TO_DROP = ["price_str", "ind_price_str", "range", "single_price", "suffix"]
df_adjusted = df_adjusted.drop(*COLS_TO_DROP)
# df_flattened.filter(col("ind_price_str").like("%Million High%")).show()



+----------+--------+-----------------------------------------+--------------+--------------+----+-----+----+----------+------------------------------------------------------------------------------------------------------------------------------+---------+--------------+------------+
|suburb    |postcode|address                                  |latitude      |longitude     |beds|baths|cars|house_type|dates                                                                                                                         |avg_price|classification|weekly_price|
+----------+--------+-----------------------------------------+--------------+--------------+----+-----+----+----------+------------------------------------------------------------------------------------------------------------------------------+---------+--------------+------------+
|Abbotsford|3067    |203/1 TURNER ST, ABBOTSFORD, ABBOTSFORD  |-37.7993152002|144.9975063002|2.0 |2.0  |1.0 |Unit/apmt |['September 2024']    

In [3]:
ddf = spark.read.parquet('../data/raw/oldlisting/oldlisting.parquet')
ddf.count()


118038